In [1]:
import sys, sklearn
print(sys.executable)
print(sklearn.__version__)


c:\miewmiew\Hackathon\Hack2\.venv\Scripts\python.exe
1.6.1


In [2]:
# เซลล์ 1: Setup
import ast
import os
import zipfile

import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score, classification_report


In [3]:
import os

# 1. ดู current working directory
print("cwd          :", os.getcwd())

# 2. ดูไฟล์/โฟลเดอร์ใน cwd
print("in cwd       :", os.listdir('.'))

# 3. ดูไฟล์/โฟลเดอร์ใน parent directory
parent = os.path.abspath(os.path.join(os.getcwd(), os.pardir))
print("parent       :", parent)
print("in parent    :", os.listdir(parent))

# 4. ดูอีกขั้นนึง (grand-parent ถ้าจำเป็น)
grand = os.path.abspath(os.path.join(parent, os.pardir))
print("grand parent :", grand)
print("in grandpar :", os.listdir(grand))


cwd          : c:\miewmiew\Hackathon\Hack2
in cwd       : ['.venv', 'crime-charges-analysis', 'crime-charges-analysis.zip', 'crime_charges_analysis.ipynb']
parent       : c:\miewmiew\Hackathon
in parent    : ['.venv', 'compare_top_bottom_stores.ipynb', 'data', 'Hack1', 'Hack2', 'q1competition.ipynb']
grand parent : c:\miewmiew
in grandpar : ['Hackathon', 'readfile.py', 'root', 'superai']


In [4]:
# เซลล์ เช็กและแตก ZIP ก่อน โหลดข้อมูล
import os, zipfile

# ชื่อไฟล์ ZIP ที่อยู่ใน cwd
zip_path = "crime-charges-analysis.zip"
# ชื่อโฟลเดอร์ที่จะเก็บข้อมูลหลังแตก
extract_dir = "crime-charges-analysis"

# ถ้ายังไม่มีโฟลเดอร์ให้แตก ZIP
if not os.path.isdir(extract_dir):
    with zipfile.ZipFile(zip_path, "r") as z:
        z.extractall(extract_dir)
    print("Extracted to:", extract_dir)
else:
    print("Directory already exists:", extract_dir)

# ตรวจดูไฟล์ภายในโฟลเดอร์
print(os.listdir(extract_dir))


Directory already exists: crime-charges-analysis
['classes.xlsx', 'dev_half.csv', 'submission.csv', 'train.csv', 'train_half.csv']


Ner

ประมาณไม่เกิน1นาที 35-40วินาที

In [5]:
# ติดตั้ง spaCy
!pip install spacy

# ดาวน์โหลดโมเดลภาษาไทย
!python -m spacy download th_core_news_sm


  Using cached spacy-3.8.2.tar.gz (1.3 MB)
  Installing build dependencies: started
  Installing build dependencies: finished with status 'error'


  error: subprocess-exited-with-error
  
  × pip subprocess to install build dependencies did not run successfully.
  │ exit code: 1
  ╰─> [65 lines of output]
      Ignoring numpy: markers 'python_version < "3.9"' don't match your environment
        Using cached setuptools-80.4.0-py3-none-any.whl.metadata (6.5 kB)
        Using cached Cython-0.29.37-py2.py3-none-any.whl.metadata (3.1 kB)
        Using cached cymem-2.0.11-cp313-cp313-win_amd64.whl.metadata (8.8 kB)
        Using cached preshed-3.0.9.tar.gz (14 kB)
        Installing build dependencies: started
        Installing build dependencies: finished with status 'done'
        Getting requirements to build wheel: started
        Getting requirements to build wheel: finished with status 'done'
        Preparing metadata (pyproject.toml): started
        Preparing metadata (pyproject.toml): finished with status 'done'
        Using cached murmurhash-1.0.12-cp313-cp313-win_amd64.whl.metadata (2.2 kB)
        Using cached thinc-8.3

In [8]:
# 1) ติดตั้ง (ถ้ายังไม่ติดตั้ง)
!pip install pythainlp torch



[notice] A new release of pip is available: 25.0.1 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


ไม่ทำnerละ ทำไม่ได้

ML baseline

In [3]:
# === เตรียม Data ===
import pandas as pd, ast
from sklearn.preprocessing import MultiLabelBinarizer

# โหลด data ที่แบ่งไว้
train_df = pd.read_csv("crime-charges-analysis/train_half.csv")
dev_df   = pd.read_csv("crime-charges-analysis/dev_half.csv")

# ฟังก์ชัน parse answers
def parse_answers(cell):
    lst = ast.literal_eval(cell)
    out = {}
    for d in lst:
        for p, cs in d.items():
            out[p] = ast.literal_eval(cs)
    return out

train_df["parsed"] = train_df["answers"].apply(parse_answers)
dev_df  ["parsed"] = dev_df  ["answers"].apply(parse_answers)

# สร้าง X, y สำหรับแต่ละ (person, story)
X_train, y_train = [], []
for _, row in train_df.iterrows():
    story = row["story"]
    for person, charges in row["parsed"].items():
        X_train.append(f"{person} [SEP] {story}")
        y_train.append(charges)

X_dev, y_dev = [], []
for _, row in dev_df.iterrows():
    story = row["story"]
    for person, charges in row["parsed"].items():
        X_dev.append(f"{person} [SEP] {story}")
        y_dev.append(charges)

# แปลง labels → binary matrix
mlb = MultiLabelBinarizer(classes=[1,2,3,4,5,6,7])
Y_train = mlb.fit_transform(y_train)
Y_dev   = mlb.transform(y_dev)

# === สร้าง & เทรน Pipeline ===
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.multiclass import OneVsRestClassifier
from sklearn.linear_model import LogisticRegression

pipeline = Pipeline([
    ("tfidf", TfidfVectorizer(ngram_range=(1,2), max_features=30000)),
    ("clf", OneVsRestClassifier(
        LogisticRegression(solver="liblinear", class_weight="balanced"), 
        n_jobs=-1))
])

pipeline.fit(X_train, Y_train)

# === ประเมินผลบน Dev ===
from sklearn.metrics import f1_score, classification_report

Y_pred = pipeline.predict(X_dev)
print("F1-score (macro):", f1_score(Y_dev, Y_pred, average="macro"))
print("\nDetailed report:\n", classification_report(Y_dev, Y_pred, target_names=[str(i) for i in mlb.classes_]))


F1-score (macro): 0.6605556701269235

Detailed report:
               precision    recall  f1-score   support

           1       0.46      0.56      0.51       307
           2       0.65      0.79      0.71       417
           3       0.63      0.71      0.66       190
           4       0.66      0.83      0.74       255
           5       0.55      0.75      0.63       373
           6       0.64      0.69      0.66       251
           7       0.59      0.89      0.71       247

   micro avg       0.59      0.75      0.66      2040
   macro avg       0.60      0.75      0.66      2040
weighted avg       0.59      0.75      0.66      2040
 samples avg       0.57      0.72      0.61      2040



c:\miewmiew\Hackathon\Hack2\.venv\Lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


เริ่มรันการวิเคราะห์ข้อมูล

In [4]:
import pandas as pd

train_df = pd.read_csv("crime-charges-analysis/train_half.csv")
dev_df   = pd.read_csv("crime-charges-analysis/dev_half.csv")

print("Train half:", train_df.shape)
print("Dev   half:", dev_df.shape)


Train half: (900, 3)
Dev   half: (900, 3)


In [5]:
import ast

def parse_answers(cell: str) -> dict[str, list[int]]:
    raw = ast.literal_eval(cell)  # list of dicts
    out = {}
    for d in raw:
        for person, charges_str in d.items():
            out[person] = ast.literal_eval(charges_str)
    return out

# สร้างคอลัมน์ใหม่
train_df["answers_parsed"] = train_df["answers"].apply(parse_answers)
dev_df["answers_parsed"]   = dev_df["answers"].apply(parse_answers)

# ตรวจสอบ
display(train_df[["id","answers","answers_parsed"]].head(3))


,id,answers,answers_parsed
0,6AzyVcLTK0,"[{'งิ่มเฮียง': '[2,5,7]'}]","{'งิ่มเฮียง': [2, 5, 7]}"
1,KVG7hHQ1L8,"[{'เจตน์ณัชนัย': '[1]'}, {'คออีก': '[5,7]'}]","{'เจตน์ณัชนัย': [1], 'คออีก': [5, 7]}"
2,7YGDGTUHqt,"[{'กำไล': '[1]'}, {'กัมปณันท์': '[2,4]'}, {'ขว...","{'กำไล': [1], 'กัมปณันท์': [2, 4], 'ขวัญชีวี':..."


In [6]:
rows = []
for _, row in train_df.iterrows():
    sid = row["id"]
    text = row["story"]
    for person, charges in row["answers_parsed"].items():
        for c in charges:
            rows.append({"id": sid, "story": text, "person": person, "charge": c})

train_long = pd.DataFrame(rows)
print("Train long:", train_long.shape)
display(train_long.head())


Train long: (2087, 4)


,id,story,person,charge
0,6AzyVcLTK0,งิ่มเฮียงขโมยกระเป๋าจากตลาดแล้วยิงฆ่าลูกค้าในร้าน,งิ่มเฮียง,2
1,6AzyVcLTK0,งิ่มเฮียงขโมยกระเป๋าจากตลาดแล้วยิงฆ่าลูกค้าในร้าน,งิ่มเฮียง,5
2,6AzyVcLTK0,งิ่มเฮียงขโมยกระเป๋าจากตลาดแล้วยิงฆ่าลูกค้าในร้าน,งิ่มเฮียง,7
3,KVG7hHQ1L8,คออีกวางแผนใส่สารหนูในขนมปังที่เจตน์ณัชนัยชอบก...,เจตน์ณัชนัย,1
4,KVG7hHQ1L8,คออีกวางแผนใส่สารหนูในขนมปังที่เจตน์ณัชนัยชอบก...,คออีก,5


In [7]:
classes = pd.read_excel("crime-charges-analysis/classes.xlsx")
display(classes)

# สร้าง dict id→list of keywords (เบื้องต้น)
crime_keywords = {
    2: ["ขโมย","ลัก","ชิงทรัพย์"],
    3: ["ประมาท","ไม่ตั้งใจ"],
    4: ["บุกรุก"],
    5: ["เจตนา","จงใจ","ทำร้าย"],
    6: ["ฉ้อโกง","ปลอมแปลง"],
    7: ["ฆ่า","ตาย","เสียชีวิต"]
}


,Class,Description
0,1,ไม่มีความผิดหรือไม่มีความเกี่ยวข้องกับเหตุการณ์
1,2,ลักทรัพย์หรือชิงทรัพย์
2,3,กระทำการใดๆด้วยความประมาทหรือละเลยส่งผลให้เกิด...
3,4,บุกรุกเข้าเขตหวงห้ามหรือนอกเวลาทำการ
4,5,กระทำการใดๆด้วยเจตนาที่ส่งผลให้เกิดความเสียหาย...
5,6,"ฉ้อโกงประชาชน, ปลอมแปลงเอกสาร หรือปลอมแปลงสิ่ง..."
6,7,ทำให้มีคนตายด้วยวิธีการใดๆก็ตาม


In [8]:
import re

def classify_text(text: str) -> dict[str, list[int]]:
    # ดึงรายชื่อจาก train set
    names = train_df["answers_parsed"].explode().index  # หรือสร้าง gazetteer list
    # เตรียม dict เปล่า
    pred = {p: [] for p in train_df["answers_parsed"].iloc[0].keys()}
    
    for sent in re.split(r"[\.!?]", text):
        sl = sent.lower()
        detected = {cls for cls, kws in crime_keywords.items() if any(kw in sl for kw in kws)}
        if 7 in detected:
            if 5 in detected: detected.add(5)
            elif 3 in detected: detected.add(3)
        for name in pred:
            if name in sent:
                pred[name].extend(detected)
    # จัด final
    for name, lst in pred.items():
        uniq = sorted(set(lst))
        pred[name] = uniq or [1]
    return pred


In [9]:
from sklearn.metrics import f1_score

y_true, y_pred = [], []
for _, row in dev_df.iterrows():
    gt = row["answers_parsed"]                # dict person→charges list
    pred = classify_text(row["story"])        # dict same format
    for person, charges in gt.items():
        for c in range(1,8):
            y_true.append(int(c in charges))
            y_pred.append(int(c in pred.get(person,[])))

print("Dev F1 (macro):", f1_score(y_true, y_pred, average="macro"))


Dev F1 (macro): 0.44154057771664373


บันทึกไฟล์ summit

In [ ]:
sample = pd.read_csv("crime-charges-analysis/submission.csv")
subs = []
for _, row in train_df.iterrows():  # แทน test.csv จริงใช้ dev_df หรือโหลด test.csv มาใช้นะ
    lst = [{p: str(ch)} for p,ch in classify_text(row["story"]).items()]
    subs.append(lst)
sample["answers"] = subs
sample.to_csv("my_submission.csv", index=False, encoding="utf-8-sig")


เพิ่มคะแนน

In [10]:
from sklearn.metrics import classification_report

print(classification_report(
    Y_dev, Y_pred,
    target_names=[str(i) for i in mlb.classes_]
))


              precision    recall  f1-score   support

           1       0.46      0.56      0.51       307
           2       0.65      0.79      0.71       417
           3       0.63      0.71      0.66       190
           4       0.66      0.83      0.74       255
           5       0.55      0.75      0.63       373
           6       0.64      0.69      0.66       251
           7       0.59      0.89      0.71       247

   micro avg       0.59      0.75      0.66      2040
   macro avg       0.60      0.75      0.66      2040
weighted avg       0.59      0.75      0.66      2040
 samples avg       0.57      0.72      0.61      2040



c:\miewmiew\Hackathon\Hack2\.venv\Lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


In [11]:
import numpy as np

probs = pipeline.predict_proba(X_dev)  # shape (n_samples, n_classes)
best_thresh = {}
for i, cls in enumerate(mlb.classes_):
    best_f1, best_t = 0, 0.5
    for t in np.linspace(0.1, 0.9, 17):
        y_pred_t = (probs[:, i] >= t).astype(int)
        f1 = f1_score(Y_dev[:, i], y_pred_t)
        if f1 > best_f1:
            best_f1, best_t = f1, t
    best_thresh[cls] = best_t
    print(f"Class {cls}: thresh={best_t:.2f}, F1={best_f1:.3f}")


Class 1: thresh=0.40, F1=0.524
Class 2: thresh=0.50, F1=0.712
Class 3: thresh=0.45, F1=0.676
Class 4: thresh=0.55, F1=0.739
Class 5: thresh=0.50, F1=0.635
Class 6: thresh=0.45, F1=0.677
Class 7: thresh=0.50, F1=0.706


In [12]:
Y_pred_thr = np.zeros_like(probs, dtype=int)
for i, cls in enumerate(mlb.classes_):
    Y_pred_thr[:, i] = (probs[:, i] >= best_thresh[cls]).astype(int)

print("F1 with tuned thresholds:", 
      f1_score(Y_dev, Y_pred_thr, average="macro"))


F1 with tuned thresholds: 0.6670658474105817


In [13]:
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.pipeline import FeatureUnion

class KeywordFlag(BaseEstimator, TransformerMixin):
    def __init__(self, keywords): self.keywords = keywords
    def fit(self, X, y=None): return self
    def transform(self, X):
        # X is list of strings "person [SEP] story"
        flags = []
        for text in X:
            story = text.split("[SEP]")[1]
            lower = story.lower()
            flags.append([1 if kw in lower else 0 for kw in self.keywords])
        return np.array(flags)

# สมมติเรา flatten keywords ทั้งหมดลง list เดียว
all_keywords = sum(crime_keywords.values(), [])
lex_union = FeatureUnion([
    ("tfidf", TfidfVectorizer(ngram_range=(1,2), max_features=20000)),
    ("lex", KeywordFlag(all_keywords))
])

hybrid = Pipeline([
    ("features", lex_union),
    ("clf", OneVsRestClassifier(LogisticRegression(solver="liblinear"), n_jobs=-1))
])
hybrid.fit(X_train, Y_train)
Yh_pred = hybrid.predict(X_dev)
print("With lex features:", f1_score(Y_dev, Yh_pred, average="macro"))


With lex features: 0.5179174402411891


In [14]:
def classify_hybrid(text):
    rule = classify_text(text)        # dict person→[classes]
    Xs   = [f"{p} [SEP] {text}" for p in rule]
    probs = pipeline.predict_proba(Xs)  # หรือ hybrid.predict_proba
    out = {}
    for idx, p in enumerate(rule):
        ml_preds = [cls for cls, prob in zip(mlb.classes_, probs[idx]) 
                    if prob >= best_thresh[cls]]
        combo = sorted(set(rule[p]) | set(ml_preds)) or [1]
        out[p] = combo
    return out


In [18]:
from sklearn.metrics import f1_score

y_hybrid_true, y_hybrid_pred = [], []

for _, row in dev_df.iterrows():
    # parse ground-truth
    gt = parse_answers(row["answers"])        # dict: person → [charges]
    # hybrid prediction
    pred = classify_hybrid(row["story"])      # dict: person → [charges]
    
    for person, true_charges in gt.items():
        # if hybrid didn’t predict this person at all, assume [1]
        pred_charges = pred.get(person, [1])
        
        for cls in mlb.classes_:             # classes = [1,2,…,7]
            y_hybrid_true.append(int(cls in true_charges))
            y_hybrid_pred.append(int(cls in pred_charges))

# now compute your F1
print("Hybrid F1:", f1_score(y_hybrid_true, y_hybrid_pred, average="macro"))


Hybrid F1: 0.5016968581991004


In [1]:
y_hybrid_true, y_hybrid_pred = [], []
for _, row in dev_df.iterrows():
    gt = parse_answers(row["answers"])
    pred = classify_hybrid(row["story"])
    for person in gt:
        for cls in mlb.classes_:
            y_hybrid_true.append(int(cls in gt[person]))
            y_hybrid_pred.append(int(cls in pred[person]))
print("Hybrid F1:", f1_score(y_hybrid_true, y_hybrid_pred, average="macro"))

NameError: name 'dev_df' is not defined

โค้ดเดียวจบ

In [3]:
from sklearn.metrics import f1_score

# กำหนด list ของคลาสตั้งแต่ 1–7
classes_list = [1,2,3,4,5,6,7]

y_hybrid_true, y_hybrid_pred = [], []

for _, row in dev_df.iterrows():
    gt = parse_answers(row["answers"])         # dict จริง
    pred = classify_hybrid(row["story"])       # dict ทำนาย

    for person, true_charges in gt.items():
        # ถ้า hybrid ไม่ทำนายชื่อนี้ ให้ถือว่าคลาส [1]
        pred_charges = pred.get(person, [1])
        for cls in classes_list:
            y_hybrid_true.append( 1 if cls in true_charges else 0 )
            y_hybrid_pred.append( 1 if cls in pred_charges else 0 )

print("Hybrid F1 (macro):", f1_score(y_hybrid_true, y_hybrid_pred, average="macro"))


Hybrid F1 (macro): 0.5016968581991004


In [9]:
# 1) โหลดและ parse dev set
import pandas as pd, ast
from sklearn.metrics import f1_score

dev_df = pd.read_csv("crime-charges-analysis/dev_half.csv")

def parse_answers(cell: str) -> dict[str, list[int]]:
    raw = ast.literal_eval(cell)
    out = {}
    for d in raw:
        for person, charges_str in d.items():
            out[person] = ast.literal_eval(charges_str)
    return out

dev_df["parsed"] = dev_df["answers"].apply(parse_answers)

# 2) ใส่ฟังก์ชัน classify_hybrid ของคุณตรงนี้
def classify_hybrid(text: str) -> dict[str, list[int]]:
    # TODO: เปลี่ยนเป็น logic Ensemble ที่คุณพัฒนาจริง
    # ตัวอย่าง dummy: ทุกคนไม่มีข้อหา ([1])
    # ให้กลับเป็น dict person → [1]
    # แต่ถ้าคุณมี rule-based/ML ผสม ให้ใส่ที่นี่เลย
    sample_persons = list(dev_df["parsed"].iloc[0].keys())
    return {p: [1] for p in sample_persons}

# 3) คำนวณ Hybrid F1
classes_list = [1,2,3,4,5,6,7]
y_true, y_pred = [], []

for _, row in dev_df.iterrows():
    gt   = row["parsed"]                 # dict ของจริง
    pred = classify_hybrid(row["story"]) # dict ทำนาย

    for person, true_charges in gt.items():
        pred_charges = pred.get(person, [1])  # default เป็น [1] ถ้าไม่มี
        for cls in classes_list:
            y_true.append(1 if cls in true_charges else 0)
            y_pred.append(1 if cls in pred_charges else 0)

print("Hybrid F1 (macro):", f1_score(y_true, y_pred, average="macro"))


Hybrid F1 (macro): 0.5016968581991004


เพิ่มคะแนน2

In [12]:
# 1) โหลดและเตรียมข้อมูล train/dev อีกครั้ง
import pandas as pd, ast
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.multiclass import OneVsRestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import f1_score
import numpy as np

# โหลดไฟล์ที่แบ่งไว้
train_df = pd.read_csv("crime-charges-analysis/train_half.csv")
dev_df   = pd.read_csv("crime-charges-analysis/dev_half.csv")

# ฟังก์ชัน parse answers
def parse_answers(cell: str) -> dict[str, list[int]]:
    raw = ast.literal_eval(cell)
    out = {}
    for d in raw:
        for p, cs in d.items():
            out[p] = ast.literal_eval(cs)
    return out

train_df["parsed"] = train_df["answers"].apply(parse_answers)
dev_df  ["parsed"] = dev_df  ["answers"].apply(parse_answers)

# สร้าง X_train, Y_train, X_dev, Y_dev
X_train, y_train = [], []
for _, row in train_df.iterrows():
    for person, charges in row["parsed"].items():
        X_train.append(f"{person} [SEP] {row['story']}")
        y_train.append(charges)

X_dev, y_dev = [], []
for _, row in dev_df.iterrows():
    for person, charges in row["parsed"].items():
        X_dev.append(f"{person} [SEP] {row['story']}")
        y_dev.append(charges)

# แปลง labels → binary matrix
mlb = MultiLabelBinarizer(classes=[1,2,3,4,5,6,7])
Y_train = mlb.fit_transform(y_train)
Y_dev   = mlb.transform(y_dev)

# 2) สร้าง ML pipeline (TF-IDF + LogisticRegression)
pipeline = Pipeline([
    ("tfidf", TfidfVectorizer(ngram_range=(1,2), max_features=30000)),
    ("clf", OneVsRestClassifier(
        LogisticRegression(solver="liblinear", class_weight="balanced"),
        n_jobs=-1
    ))
])
pipeline.fit(X_train, Y_train)

# 3) คำนวณ best threshold per class บน dev set
probs = pipeline.predict_proba(X_dev)  # ค่า probability shape = (n_samples, 7)
best_thresh = {}
for i, cls in enumerate([1,2,3,4,5,6,7]):
    best_f1, best_t = 0, 0.5
    for t in np.linspace(0.1, 0.9, 17):
        y_t = (probs[:, i] >= t).astype(int)
        f1 = f1_score(Y_dev[:, i], y_t)
        if f1 > best_f1:
            best_f1, best_t = f1, t
    best_thresh[cls] = best_t
    print(f"Class {cls}: best threshold = {best_t:.2f} with F1 = {best_f1:.3f}")

print("\nThreshold dictionary:", best_thresh)

Class 1: best threshold = 0.40 with F1 = 0.524
Class 2: best threshold = 0.50 with F1 = 0.712
Class 3: best threshold = 0.45 with F1 = 0.676
Class 4: best threshold = 0.55 with F1 = 0.739
Class 5: best threshold = 0.50 with F1 = 0.635
Class 6: best threshold = 0.45 with F1 = 0.677
Class 7: best threshold = 0.50 with F1 = 0.706

Threshold dictionary: {1: np.float64(0.4), 2: np.float64(0.5), 3: np.float64(0.45000000000000007), 4: np.float64(0.55), 5: np.float64(0.5), 6: np.float64(0.45000000000000007), 7: np.float64(0.5)}


In [13]:
# คำนวณ best_thresh จาก dev set (รันครั้งเดียว)
import numpy as np
probs = pipeline.predict_proba(X_dev)  # X_dev = [f"{person} [SEP] {story}"…]
best_thresh = {}
for i, cls in enumerate([1,2,3,4,5,6,7]):
    best_f1, best_t = 0, .5
    for t in np.linspace(0.1,0.9,17):
        y_t = (probs[:,i]>=t).astype(int)
        f1 = f1_score(Y_dev[:,i], y_t)
        if f1>best_f1:
            best_f1, best_t = f1, t
    best_thresh[cls] = best_t


In [14]:
# 1) โหลดและเตรียมข้อมูล train/dev อีกครั้ง
import pandas as pd, ast
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.multiclass import OneVsRestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import f1_score
import numpy as np

# โหลดไฟล์ที่แบ่งไว้
train_df = pd.read_csv("crime-charges-analysis/train_half.csv")
dev_df   = pd.read_csv("crime-charges-analysis/dev_half.csv")

# ฟังก์ชัน parse answers
def parse_answers(cell: str) -> dict[str, list[int]]:
    raw = ast.literal_eval(cell)
    out = {}
    for d in raw:
        for p, cs in d.items():
            out[p] = ast.literal_eval(cs)
    return out

train_df["parsed"] = train_df["answers"].apply(parse_answers)
dev_df  ["parsed"] = dev_df  ["answers"].apply(parse_answers)

# สร้าง X_train, Y_train, X_dev, Y_dev
X_train, y_train = [], []
for _, row in train_df.iterrows():
    for person, charges in row["parsed"].items():
        X_train.append(f"{person} [SEP] {row['story']}")
        y_train.append(charges)

X_dev, y_dev = [], []
for _, row in dev_df.iterrows():
    for person, charges in row["parsed"].items():
        X_dev.append(f"{person} [SEP] {row['story']}")
        y_dev.append(charges)

# แปลง labels → binary matrix
mlb = MultiLabelBinarizer(classes=[1,2,3,4,5,6,7])
Y_train = mlb.fit_transform(y_train)
Y_dev   = mlb.transform(y_dev)

# 2) สร้าง ML pipeline (TF-IDF + LogisticRegression)
pipeline = Pipeline([
    ("tfidf", TfidfVectorizer(ngram_range=(1,2), max_features=30000)),
    ("clf", OneVsRestClassifier(
        LogisticRegression(solver="liblinear", class_weight="balanced"),
        n_jobs=-1
    ))
])
pipeline.fit(X_train, Y_train)

# 3) คำนวณ best threshold per class บน dev set
probs = pipeline.predict_proba(X_dev)  # ค่า probability shape = (n_samples, 7)
best_thresh = {}
for i, cls in enumerate([1,2,3,4,5,6,7]):
    best_f1, best_t = 0, 0.5
    for t in np.linspace(0.1, 0.9, 17):
        y_t = (probs[:, i] >= t).astype(int)
        f1 = f1_score(Y_dev[:, i], y_t)
        if f1 > best_f1:
            best_f1, best_t = f1, t
    best_thresh[cls] = best_t
    print(f"Class {cls}: best threshold = {best_t:.2f} with F1 = {best_f1:.3f}")

print("\nThreshold dictionary:", best_thresh)

Class 1: best threshold = 0.40 with F1 = 0.524
Class 2: best threshold = 0.50 with F1 = 0.712
Class 3: best threshold = 0.45 with F1 = 0.676
Class 4: best threshold = 0.55 with F1 = 0.739
Class 5: best threshold = 0.50 with F1 = 0.635
Class 6: best threshold = 0.45 with F1 = 0.677
Class 7: best threshold = 0.50 with F1 = 0.706

Threshold dictionary: {1: np.float64(0.4), 2: np.float64(0.5), 3: np.float64(0.45000000000000007), 4: np.float64(0.55), 5: np.float64(0.5), 6: np.float64(0.45000000000000007), 7: np.float64(0.5)}
